# HIPAA Compliance & PII Masking

## Business Scenario
A healthcare provider is ingesting patient medical records into their Data Lakehouse. These records contain **Protected Health Information (PHI)** and **Personally Identifiable Information (PII)** such as names, SSNs, and emails.

The organization has two strict requirements:
1. **Governance Consistency**: Every medical dataset must adhere to the institutional HIPAA policy (SSN formats, Email validation).
2. **Risk Mitigation**: All PII must be masked automatically before it reaches the 'Silver' layer (analytics ready), ensuring data scientists can work with the data without exposure to raw identifiers.

## Value Proposition
- **Zero-Day Compliance**: Apply shared HIPAA policy packs across hundreds of tables instantly.
- **Automated Protection**: Shift-left security by masking PII during ingestion, not as a downstream afterthought.
- **Audit Ready**: Every record is tagged with lineage, and failed records are safely quarantined for compliance review.


## 1. Setup
We'll use the `DataProcessor` and point it to our contract. We'll also tell it to use the `hipaa_compliance` Policy Pack we created earlier.

In [ ]:
import os
import polars as pl
from pathlib import Path
from lakelogic import DataProcessor

# Set paths

def resolve_example_dir(name: str) -> Path:
    cwd = Path.cwd()
    for base in [cwd] + list(cwd.parents):
        candidates = [
            base / name,
            base / "examples" / "08_compliance_governance" / name,
            base / "lakelogic" / "examples" / "08_compliance_governance" / name,
        ]
        for candidate in candidates:
            if candidate.exists():
                return candidate
    return cwd / name


def resolve_repo_root(start: Path) -> Path:
    for base in [start] + list(start.parents):
        if (base / "policy_packs" / "hipaa_compliance.yaml").exists():
            return base
    return start


base_path = resolve_example_dir("hipaa_pii_masking")
repo_root = resolve_repo_root(base_path)

contract_path = base_path / "contracts" / "medical_records.yaml"
policy_pack_path = repo_root / "policy_packs" / "hipaa_compliance.yaml"
data_path = base_path / "data" / "phi_records.csv"

print(f"Contract: {contract_path.resolve()}")
print(f"Policy Pack: {policy_pack_path.resolve()}")
print(f"Data: {data_path.resolve()}")


## 2. Load the HIPAA Policy Pack
In LakeLogic, a **Policy Pack** acts as a baseline for all contracts. It enforces rules even if they aren't explicitly written in the individual table contract.

In [ ]:
import yaml
with open(policy_pack_path, "r") as f:
    policy_pack = yaml.safe_load(f)

print("Rules in HIPAA Policy Pack:")
for rule in policy_pack.get('quality', {}).get('row_rules', []):
    print(f" - {rule.get('name')}: {rule.get('description')}")

## 3. Run the Ingestion with PII Masking
The contract is configured to use the `pii_masking` hook. This hook uses NLP (via Microsoft Presidio) to find and replace sensitive data.

In [ ]:
# Initialize processor (Auto-discovered engine, e.g., Polars)
processor = DataProcessor(contract=contract_path)

# Run the source file (which contains raw PII)
source_df, good_df, bad_df = processor.run_source(data_path)

print("Run Complete!")

## 4. Inspect the Quarantine (Governance Gating)
Note that 'Bob' had an invalid email format and 'Edward' had a malformed SSN. They were quarantined by the shared HIPAA rules.

In [ ]:
print("🚨 QUARANTINE (Failed HIPAA Rules):")
print(bad_df)

## 5. Inspect the Silver Layer (Masked PII)
For the records that passed validation (Alice, Charlie, Diane), notice that their names, SSNs, and emails have been replaced with `[PROTECTED]`.

This data is now safe for downstream consumption by analytics teams.

In [ ]:
print("✅ SILVER LAYER (Clean & Masked):")
print(good_df)

## Summary
By combining **Policy Packs** with **External Logic Hooks**, LakeLogic allows organizations to turn compliance from a manual checklist into an automated gate. 

Data Engineers only need to define the table schema; the compliance team manages the policy pack, ensuring every record is protected before it ever hits a dashboard.